In [1]:
"""
Created on Mon Feb 12 17:31:03 2018
This script is used to perform a unidirectional window slide on 3D geological data and extract statistical features from each block.
Parameters: Window size, Overlap and cluster gthe blocks and then generate state tansition matrix
@author: GMohaar

Steps: 
1) Sort along a single direction where you would like to slide the window
2) Read window size and overlap settings from config file
3) Slide window and from each block , calculate statistical features
4) Return feature vector with block number as Id.    
5) Cluster all the feature vectors to n clusters (select in config file
6) Assign cluster numbers to all blocks
7) Create state transition matrix
"""

# -*- coding: utf-8 -*-
"""
Created on Mon Jan 29 09:16:08 2018

@author: GMohaar
"""
import sys
sys.modules[__name__].__dict__.clear()
import pandas as pd
import yaml
import os
from scipy.stats import kurtosis
from scipy.stats import skew

In [2]:
#read data and windowing settings from YML
with open("config.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)
file_name = cfg['filepath']
window_size = cfg['window_size']
overlap = cfg['overlap']
num_clusters = cfg['num_clusters'] 
data  = pd.read_csv(file_name,delim_whitespace=True)
sort_along = cfg['slide_along'] #bydefault set to east. go to config file if want to slide along other directions.


C:\softwares\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
'''preprocess data
1. Delete variables which are not required
2. Delete rows with incomeplte information
'''
data = data.drop(['ID','BLAST', 'BHS','FAULT'], axis = 1)
data = data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
print(data.head())

      NORTH      EAST     ELEV   CUT   ASCU   RT CODE
0  22029.21  93472.17  3080.32  0.79  0.500  9.0  LXX
1  22110.19  93681.08  3091.64  3.42  0.390  3.0  SSE
2  22011.80  93739.40  3092.10  2.36  0.320  2.0  SSE
3  22020.60  93478.04  3092.63  0.23  0.001  9.0  LXX
4  22013.10  93474.66  3093.09  0.26  0.001  9.0  LXX


In [4]:
'''Import windows slider class'''
from Window_slide import Window_slide 

In [5]:
# Defining Function to extract features: calculates kurtosis, mean, skewness, st dev of different attributtes
def extract_features(window_matrix):    
     rtypes =  window_matrix.RT.nunique()
     ctypes = window_matrix.CODE.nunique()
     kurtosis_cut =  kurtosis(window_matrix.CUT)
     kurtosis_ascu = kurtosis(window_matrix.ASCU)
     skewness_cut = skew(window_matrix.CUT)
     skewness_ascu = skew(window_matrix.ASCU)
     mean_north = window_matrix.NORTH.mean()
     mean_east = window_matrix.EAST.mean()
     mean_elev = window_matrix.ELEV.mean()
     mean_cut = window_matrix.CUT.mean()
     mean_ascu = window_matrix.ASCU.mean()
     std_ascu = window_matrix.ASCU.std()
     std_cut = window_matrix.CUT.std()
     std_north = window_matrix.NORTH.std()
     std_east = window_matrix.EAST.std()
     std_elev = window_matrix.ELEV.std()
     df = pd.DataFrame({'rock_types': [rtypes], 'code_types': [ctypes],'kurtosis_CUT':[kurtosis_cut], 'kurtosis_ASCU': [kurtosis_ascu], 'skewness_CUT': [skewness_cut], 'skewness_ASCU':[skewness_ascu],'mean_north': [mean_north] ,'mean_east': [mean_east],'mean_elev': [mean_elev],'mean_cut': [mean_cut],'mean_ascu': [mean_ascu],'std_north': [std_north],'std_east': [std_east],'std_elev': [std_elev],'std_cut': [std_cut],'std_ascu': [std_ascu] })      
     return(df)

In [6]:
'''Execution logic goes below
1. Window slides along the direction specified in config file
2. function call for each block to calculate features and append to data frame
3. End when last window readched
'''
#Data should be sorted in one direction either x,y,z
sorted_data = data.sort_values(sort_along)
#initialise class : window
window = Window_slide(sorted_data,window_size,overlap)
features_single_block = pd.DataFrame();
while True:
    window_matrix = window.slide()
    if len(window_matrix ==window_size):
        #print("TO BE DEVELOPED ...extracting features for block i .... Printing data in block" ,window_matrix )
        features_single_block = features_single_block.append(extract_features(window_matrix), ignore_index=True)
        if window.reached_end_of_list(): break

In [7]:
#Discard rows again with missing freatures
feature_set = pd.DataFrame(features_single_block.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False))
print(feature_set.head())

   rock_types  code_types  kurtosis_CUT  kurtosis_ASCU  skewness_CUT  \
0           9           4      7.052115      16.060144      2.548896   
1           9           4      6.517871      17.365461      2.432630   
2           7           3      5.465062       1.148718      2.225556   
3           7           3      5.440773       1.502412      2.219846   
4           8           4      6.078876       4.072747      2.316742   

   skewness_ASCU  mean_north   mean_east  mean_elev  mean_cut  mean_ascu  \
0       3.261452  22000.9585  93291.3596  3007.4548   0.66713    0.05204   
1       3.397347  21994.4186  93262.4155  3012.6817   0.70342    0.05179   
2       1.367969  21981.7750  93242.7717  3016.2124   0.74661    0.04851   
3       1.468225  21962.3940  93231.9077  3017.7024   0.76006    0.04800   
4       1.928118  21943.4862  93223.0564  3019.1894   0.74359    0.05124   

    std_north    std_east   std_elev   std_cut  std_ascu  
0  112.290425  113.334815  18.347087  0.671126  0.0

In [8]:
'''
Cluster all the blocks
Track cluster assignments along the sorted direction
'''
###k means clustering
from sklearn.cluster import KMeans
#change number of clusters in config file, current setting is 10  
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(feature_set)
state = pd.DataFrame(kmeans.labels_)
cluster_result = pd.concat([feature_set, state], axis=1)
print(cluster_result.head())

   rock_types  code_types  kurtosis_CUT  kurtosis_ASCU  skewness_CUT  \
0           9           4      7.052115      16.060144      2.548896   
1           9           4      6.517871      17.365461      2.432630   
2           7           3      5.465062       1.148718      2.225556   
3           7           3      5.440773       1.502412      2.219846   
4           8           4      6.078876       4.072747      2.316742   

   skewness_ASCU  mean_north   mean_east  mean_elev  mean_cut  mean_ascu  \
0       3.261452  22000.9585  93291.3596  3007.4548   0.66713    0.05204   
1       3.397347  21994.4186  93262.4155  3012.6817   0.70342    0.05179   
2       1.367969  21981.7750  93242.7717  3016.2124   0.74661    0.04851   
3       1.468225  21962.3940  93231.9077  3017.7024   0.76006    0.04800   
4       1.928118  21943.4862  93223.0564  3019.1894   0.74359    0.05124   

    std_north    std_east   std_elev   std_cut  std_ascu  0  
0  112.290425  113.334815  18.347087  0.671126  

In [9]:
import pandas as pd
transitions = list(kmeans.labels_)
df = pd.DataFrame(columns = ['state', 'next_state'])
for i, val in enumerate(transitions[:-1]): # We don't care about last state
    df_stg = pd.DataFrame(index=[0])
    df_stg['state'], df_stg['next_state'] = transitions[i], transitions[i+1]
    df = pd.concat([df, df_stg], axis = 0)
cross_tab = pd.crosstab(df['state'], df['next_state'])
print(cross_tab.div(cross_tab.sum(axis=1), axis=0))

next_state         0         1         2         3         4         5
state                                                                 
0           0.942019  0.000000  0.040246  0.000000  0.000000  0.017735
1           0.000000  0.911544  0.006997  0.039980  0.024988  0.016492
2           0.018291  0.005980  0.942666  0.026029  0.000000  0.007035
3           0.000685  0.027416  0.024332  0.946539  0.000000  0.001028
4           0.000000  0.029126  0.000000  0.000000  0.957282  0.013592
5           0.019355  0.022581  0.012258  0.001290  0.010968  0.933548
